In [117]:
import numpy as np
import pandas as pd
import torch
import warnings

from gensim.models import word2vec, FastText
import pandas as pd
import re
#import seaborn as sns
#import matplotlib.pyplot as plt

from torch import nn, optim, cuda
from torch.nn import functional as F
from torch.utils import data as torch_data
from sklearn import metrics
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Input, LSTM,Flatten
from keras.layers.core import Dense, Activation, Dropout
from keras.layers import Softmax
from tqdm import tqdm
from sklearn.metrics import classification_report
from keras.preprocessing import sequence
from sklearn.preprocessing import OneHotEncoder
from keras.layers.core import Dense, Activation, Dropout
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding

import pandas as pd
import numpy as np
import os

#fig_size = plt.rcParams['figure.figsize']
device = 'cuda' if cuda.is_available() else 'cpu'
if device == 'cuda':
    print(f"Torch using {cuda.get_device_name()}")
else:
    print(f"Torch using {device}")

warnings.filterwarnings('ignore')
torch.manual_seed(0)
np.random.seed(0)

#%matplotlib inline

Torch using cpu


In [118]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [146]:
male_file = r'/content/drive/MyDrive/BigDA/names/male.txt'
female_file = r'/content/drive/MyDrive/BigDA/names/female.txt'

In [147]:
male_file

'/content/drive/MyDrive/BigDA/names/male.txt'

In [148]:
names_df = pd.DataFrame(columns=["name", "gender"])

with open(male_file, mode='r') as f:
    for line in f:
        name = line[:-1] if line.endswith('\n') else line        
        names_df.loc[names_df.shape[0], :] = (name, 1)

with open(female_file, mode='r') as f:
    for line in f:
        name = line[:-1] if line.endswith('\n') else line        
        names_df.loc[names_df.shape[0], :] = (name, 0)

names_df.name = names_df.name.str.lower()
puncts = "' -"
table = str.maketrans(dict.fromkeys(puncts))
names_df.name = names_df.name.str.translate(table)

names_df

,name,gender
0,aamir,1
1,aaron,1
2,abbey,1
3,abbie,1
4,abbot,1
...,...,...
7939,zorine,0
7940,zsazsa,0
7941,zsazsa,0
7942,zulema,0


In [149]:
from sklearn.utils import shuffle
df = shuffle(names_df)
df

,name,gender
6823,orly,0
4336,donny,0
5118,hillary,0
7876,winne,0
6084,luisa,0
...,...,...
7930,zoe,0
1582,lucio,1
5167,imogen,0
6979,ralina,0


In [150]:
count = df.pivot_table(index = ['gender'], aggfunc ='size')
print(count)

#0 is female 
#1 is male



gender
0    5001
1    2943
dtype: int64


In [151]:
data = [(x, y) for x, y in zip(df['name'], df['gender'])]
data

[('orly', 0),
 ('donny', 0),
 ('hillary', 0),
 ('winne', 0),
 ('luisa', 0),
 ('tiler', 1),
 ('melisandra', 0),
 ('merlin', 1),
 ('ethel', 0),
 ('kiele', 0),
 ('faith', 0),
 ('kelsey', 1),
 ('terrijo', 0),
 ('page', 0),
 ('sharra', 0),
 ('sue', 0),
 ('franni', 0),
 ('adeline', 0),
 ('zahara', 0),
 ('netty', 0),
 ('davide', 1),
 ('cherri', 0),
 ('konstance', 0),
 ('stanly', 1),
 ('abbot', 1),
 ('rena', 0),
 ('joao', 1),
 ('antonietta', 0),
 ('horst', 1),
 ('carey', 0),
 ('diana', 0),
 ('hezekiah', 1),
 ('merrick', 1),
 ('tome', 1),
 ('myrtle', 0),
 ('twila', 0),
 ('tonya', 0),
 ('elnora', 0),
 ('loria', 0),
 ('vassily', 1),
 ('alejandra', 0),
 ('tisha', 0),
 ('clem', 0),
 ('cyrill', 1),
 ('heinz', 1),
 ('alix', 1),
 ('corky', 1),
 ('reeba', 0),
 ('claretta', 0),
 ('clemmie', 0),
 ('cornie', 0),
 ('kelli', 0),
 ('wilow', 0),
 ('sadie', 0),
 ('jimmie', 1),
 ('robbyn', 0),
 ('cherice', 0),
 ('theresina', 0),
 ('lanny', 0),
 ('elbertine', 0),
 ('helen', 0),
 ('lucius', 1),
 ('deloria', 0),
 

**Reusing the vectorize code from 3.5-classifying-movie-reviews**

In [152]:
## First we need to convert strings to letters
data1 = [(list(X.lower()),y) for (X,y) in data]
print(data1[40])

(['a', 'l', 'e', 'j', 'a', 'n', 'd', 'r', 'a'], 0)


In [153]:
corpus_train = []
for col in df.name:
   corpus_train.append(col)
corpus_train

['orly',
 'donny',
 'hillary',
 'winne',
 'luisa',
 'tiler',
 'melisandra',
 'merlin',
 'ethel',
 'kiele',
 'faith',
 'kelsey',
 'terrijo',
 'page',
 'sharra',
 'sue',
 'franni',
 'adeline',
 'zahara',
 'netty',
 'davide',
 'cherri',
 'konstance',
 'stanly',
 'abbot',
 'rena',
 'joao',
 'antonietta',
 'horst',
 'carey',
 'diana',
 'hezekiah',
 'merrick',
 'tome',
 'myrtle',
 'twila',
 'tonya',
 'elnora',
 'loria',
 'vassily',
 'alejandra',
 'tisha',
 'clem',
 'cyrill',
 'heinz',
 'alix',
 'corky',
 'reeba',
 'claretta',
 'clemmie',
 'cornie',
 'kelli',
 'wilow',
 'sadie',
 'jimmie',
 'robbyn',
 'cherice',
 'theresina',
 'lanny',
 'elbertine',
 'helen',
 'lucius',
 'deloria',
 'emilia',
 'spud',
 'yovonnda',
 'vibhu',
 'giuseppe',
 'joselyn',
 'vivien',
 'salem',
 'dana',
 'minny',
 'moira',
 'garret',
 'shirlee',
 'cheri',
 'sephira',
 'dan',
 'adrien',
 'zarla',
 'rawley',
 'parker',
 'trip',
 'tania',
 'si',
 'nicky',
 'emalia',
 'barbara',
 'magnus',
 'arleta',
 'davis',
 'milissent

In [154]:
names = []
labels = []

names = [vname for vname,x  in data1]
labels = [label for x , label in data1]
    
print(names[0:5])
print(labels[0:5])
len(names), len(labels)

[['o', 'r', 'l', 'y'], ['d', 'o', 'n', 'n', 'y'], ['h', 'i', 'l', 'l', 'a', 'r', 'y'], ['w', 'i', 'n', 'n', 'e'], ['l', 'u', 'i', 's', 'a']]
[0, 0, 0, 0, 0]


(7944, 7944)

**Converting the names into One_hot encodingt**

In [155]:
chars = ''
for name in names:
    for s in name:
        chars += s
charsu = set(chars)
print(charsu)
vocab_size = len(charsu)
print('total chars:', len(charsu))
char_indices = dict((c, i) for i, c in enumerate(charsu))
indices_char = dict((i, c) for i, c in enumerate(charsu))

indices_char[8]

{'z', 'u', 'g', 'd', 'f', 'q', 'p', 'k', 't', 's', 'c', 'h', 'r', 'n', 'l', 'j', 'i', 'm', 'x', 'o', 'b', 'e', 'w', 'y', 'a', 'v'}
total chars: 26


't'

In [156]:
def vectorize_names(names, verbose=False):
    vnames =[]
    for name in names:
        x = [char_indices[c] for c in name]
        if verbose:
            print(x)
        vx = np.eye(len(char_indices))[x]
        if verbose:
            print(vx[0:12])
        vnames.append(vx)
    return vnames

vectorize_names(names[0:2], verbose=True)

vnames = vectorize_names(names,verbose=False)

[19, 12, 14, 23]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
  0. 0.]]
[3, 19, 13, 13, 23]
[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
  0. 0.]]


In [157]:
from keras.preprocessing.sequence import pad_sequences

maxlen =20

pvnames =  pad_sequences(vnames, maxlen=maxlen)

print(len(pvnames[1]))

pvnames[1]

20


array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

**Splitting the data into train_data and test_data**

In [158]:
# Our vectorized training data
x_train = pvnames[1000:]
# Our vectorized test data
x_test = pvnames[:1000]

print(x_train.shape)

(6944, 20, 26)


In [168]:
import keras
from keras.utils.np_utils import to_categorical
y_ = to_categorical(labels) # y: class vector to be converted into a matrix (integers from 0 to num_classes). 
#print('y_.shape: ',y_.shape )  
print(y_[5:26])

[[0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]]


In [170]:
for i in range(len(y_pred)):
    if y_pred[i][0] > 0.5:
        y_pred[i][0] = 1
    else:
        y_pred[i][0] = 0
y_train = y_[1000:]
y_test = y_[:1000]
y_train.shape

(6944, 2)

Task 1: Create a convolutional neural network model with two 1D convolutional layers. Use sigmoid activation and apply a dropout of 0.3 after creating the convolution layers. In the end, create a softmax layer. Predict whether the names in test data are male or female. Evaluate your predictions (you can use the scikit learn classification report).

In [171]:
print(x_train.shape)

(6944, 20, 26)


In [172]:


inputs = Input(shape=(20, 26), name='input', dtype='float32')

print('inputs.shape: ',inputs.shape)


inputs.shape:  (None, 20, 26)


In [173]:
nb_filter = 6
dense_outputs = 10
kernel_sizes = [2, 5, 3, 3, 4, 3]
n_out = 2
batch_size = 80
nb_epochs = 50

conv = Conv1D(filters=nb_filter, kernel_size=kernel_sizes[0],
                     padding='same', activation='sigmoid',
                     input_shape=(20, 26))(inputs)
conv1 = MaxPooling1D(pool_size=5)(conv)

conv2 = Conv1D(filters=nb_filter, kernel_size=kernel_sizes[2],
                      padding='same', activation='sigmoid')(conv1)

conv5 = Flatten()(conv2)

z = Dropout(0.3)(Dense(dense_outputs, activation='sigmoid')(conv5))

pred = Dense(n_out, activation='softmax', name='output')(z)

model = Model(inputs=inputs, outputs=pred)


model.compile(loss='categorical_crossentropy', optimizer='rmsprop',
              metrics=['accuracy'])
model.summary()


Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 20, 26)]          0         
                                                                 
 conv1d_24 (Conv1D)          (None, 20, 6)             318       
                                                                 
 max_pooling1d_14 (MaxPoolin  (None, 4, 6)             0         
 g1D)                                                            
                                                                 
 conv1d_25 (Conv1D)          (None, 4, 6)              114       
                                                                 
 flatten_11 (Flatten)        (None, 24)                0         
                                                                 
 dense_20 (Dense)            (None, 10)                250       
                                                          

In [174]:
history=model.fit(x_train, y_train, batch_size=464,
           epochs=nb_epochs, validation_split=0.2, verbose=True)

Epoch 1/50
12/12 [==============================] - 1s 40ms/step - loss: 0.8789 - accuracy: 0.4130 - val_loss: 0.7443 - val_accuracy: 0.4032
Epoch 2/50
12/12 [==============================] - 0s 17ms/step - loss: 0.7842 - accuracy: 0.4653 - val_loss: 0.7015 - val_accuracy: 0.4032
Epoch 3/50
12/12 [==============================] - 0s 16ms/step - loss: 0.7474 - accuracy: 0.4950 - val_loss: 0.6808 - val_accuracy: 0.5968
Epoch 4/50
12/12 [==============================] - 0s 15ms/step - loss: 0.7187 - accuracy: 0.5357 - val_loss: 0.6745 - val_accuracy: 0.5968
Epoch 5/50
12/12 [==============================] - 0s 13ms/step - loss: 0.7048 - accuracy: 0.5651 - val_loss: 0.6752 - val_accuracy: 0.5968
Epoch 6/50
12/12 [==============================] - 0s 14ms/step - loss: 0.6954 - accuracy: 0.5883 - val_loss: 0.6769 - val_accuracy: 0.5968
Epoch 7/50
12/12 [==============================] - 0s 16ms/step - loss: 0.6980 - accuracy: 0.5770 - val_loss: 0.6780 - val_accuracy: 0.5968
Epoch 8/50
12

In [175]:
y_pred = model.predict(x_test)
for i in range(len(y_pred)):
    if y_pred[i][0] > 0.5:
        y_pred[i][0] = 1
    else:
        y_pred[i][0] = 0
        
target_names = ['Male', 'Female']
print(classification_report(y_test, y_pred, target_names=target_names))

ValueError: ignored

In [94]:
score = model.evaluate(x_test,y_test)
print('model accuracy is:',score[1]*100,'percentage')

32/32 [==============================] - 0s 2ms/step - loss: 0.6702 - accuracy: 0.6080
model accuracy is: 60.79999804496765 percentage


In [107]:
model = Sequential()

model.add(LSTM(32, dropout=0.2, input_shape=(20, 26),recurrent_dropout=0.2))
model.add(Dense(2, activation='sigmoid'))




model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(x_train, y_train, epochs=50, batch_size=256,verbose = 1,validation_data=(x_test, y_test))

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 32)                7552      
                                                                 
 dense_12 (Dense)            (None, 2)                 66        
                                                                 
Total params: 7,618
Trainable params: 7,618
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/50
28/28 [==============================] - 5s 59ms/step - loss: 0.6710 - accuracy: 0.6195 - val_loss: 0.6591 - val_accuracy: 0.6080
Epoch 2/50
28/28 [==============================] - 1s 38ms/step - loss: 0.6396 - accuracy: 0.6326 - val_loss: 0.6464 - val_accuracy: 0.6080
Epoch 3/50
28/28 [==============================] - 1s 38ms/step - loss: 0.6253 - accuracy: 0.6328 - val_loss: 0.6299 - val_accuracy: 0.6080
Epoch 4/50
28/28 [==

In [108]:
score1 = model.evaluate(x_test,y_test)
print('model accuracy is:',score1[1]*100,'percentage')

32/32 [==============================] - 0s 7ms/step - loss: 0.4260 - accuracy: 0.8060
model accuracy is: 80.59999942779541 percentage


In [113]:
model = Sequential()

model.add(Conv1D(filters=32, input_shape=(20, 26),kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))
model.add(LSTM(32, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(x_train, y_train, epochs=50, batch_size=256,verbose = 1,validation_data=(x_test, y_test))

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_17 (Conv1D)          (None, 20, 32)            2528      
                                                                 
 max_pooling1d_10 (MaxPoolin  (None, 10, 32)           0         
 g1D)                                                            
                                                                 
 dropout_10 (Dropout)        (None, 10, 32)            0         
                                                                 
 lstm_10 (LSTM)              (None, 32)                8320      
                                                                 
 dense_16 (Dense)            (None, 2)                 66        
                                                                 
Total params: 10,914
Trainable params: 10,914
Non-trainable params: 0
_________________________________________________

In [114]:
score2 = model.evaluate(x_test,y_test)
print('model accuracy is:',score2[1]*100,'percentage')

32/32 [==============================] - 0s 3ms/step - loss: 0.3936 - accuracy: 0.8160
model accuracy is: 81.5999984741211 percentage


In [116]:
y_pred = model.predict(x_test)
target_names = ['Male', 'Female']
print(classification_report(y_test, y_pred, target_names=target_names))

ValueError: ignored